# Start EVA Server

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Launch EVA server

We use this notebook for launching the EVA server.

In [1]:
## Install EVA package if needed
%pip install "evadb" --quiet

import re
import os
import time
import platform
import itertools

from psutil import process_iter
from signal import SIGTERM


def is_eva_server_process(proc):
    if platform.system() == "Darwin":
        if proc.name() == "Python":
            cmdline = proc.cmdline()
            return len(cmdline) > 1 and "eva_server" in cmdline[1]
    elif platform.system() == "Linux":
        return proc.name() == "eva_server"
    
    # TODO: Handle for Windows
    return False

def shell(command):
    print(command)
    os.system(command)

def stop_eva_server():
    for proc in process_iter():
        if is_eva_server_process(proc):
            print("Stopping EVA Server ...")
            proc.send_signal(SIGTERM)

def is_eva_server_running():
    for proc in process_iter():
        if is_eva_server_process(proc):
            return True
    return False

def launch_eva_server():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    # Start EVA server
    print("Starting EVA Server ...")
    shell("nohup eva_server > eva.log 2>&1 &")

    last_few_lines_count = 3
    try:
        with open('eva.log', 'r') as f:
            for lines in itertools.zip_longest(*[f]*last_few_lines_count):
                print(lines)
    except FileNotFoundError:
        pass

    # Wait for server to start
    time.sleep(10)

def connect_to_server():
    from eva.server.db_api import connect
    import nest_asyncio
    nest_asyncio.apply()

    status = is_eva_server_running()
    if status:
        stop_eva_server()
    
    launch_eva_server()

    # Connect client with server
    connection = connect(host="127.0.0.1", port=8803) 
    cursor = connection.cursor()

    return cursor

Note: you may need to restart the kernel to use updated packages.
nohup eva_server --port 8803 > eva.log 2>&1 &
